## Notes before running this script
- Close all the Excel files you want to convert to CSV before running this code.
- This script will convert all of your files with .xlsx or .xls located in the path you provide.

## To run this script 
 - Execute every cell containing code by clicking the cell and pressing Shift + Enter.

## Enter directory with all the excel files you want to convert.

In [ ]:
import pandas as pd
from datetime import date
import os

root_dir = input()

In [ ]:
def many_as_one (root_dir):

    # 1: List all excel files in directory. 

    excel_file_list = []
    try:
        for file in os.listdir(root_dir):
            if file.endswith(".xlsx") or file.endswith(".xls") :
                excel_file_list.append(file)
        print("Number of excel files to be processed:", len(excel_file_list))

    except FileNotFoundError:
        print("Incorrect file path")
    
    
    # 2: Make new directory for csv files that will be loaded into the database.

    today = date.today().strftime("%d%b%Y")
    csv_folder_name= "{}{}".format(today.upper(), "_CSV")

    try:
        new_csv_folder_path = "{}\{}".format(root_dir, csv_folder_name)
        if not os.path.exists(new_csv_folder_path):
            os.mkdir(new_csv_folder_path)
    except OSError:
        print ("Creation of the directory %s failed" % new_csv_folder_path)
    else:
        print ("Directory for CSV files created: %s " % new_csv_folder_path)


    # 3. Get a list of dictionaries {K:V = tab name : df of tab contents}
    list_of_dict = []
    l = 0
    for file in excel_file_list:
        dict_of_df = pd.read_excel ('{}\{}'.format(root_dir, excel_file_list[l]), sheet_name = None)  
        l+=1
        list_of_dict.append(dict_of_df)

    
    if len(list_of_dict) == 0:
        print("\nThere are no Excel files in the directory you provided." + "\n"
             + "(Hint: you may be mistaking CSV files for Excel files. It happens to the best of us :)")
    else:
        # 4. Generate a list of tab names 
        list_of_tabs = list(dict_of_df.keys())
        print("List of tabs:", list_of_tabs)

        try:
            t = 0
            while t < len(list_of_tabs):
                tab_iterator = list_of_tabs[t]
                t+=1
                x = 0

                #define a combined_df outside the inner loop so that it doesn't get redefined at every iteration
                dict_boy = list_of_dict[x]
                temp_df = dict_boy.get(tab_iterator) 
                combined_df = pd.DataFrame(columns=(temp_df.columns))

                while x < len(list_of_dict):
                    dict_boy = list_of_dict[x]
                    temp_df = dict_boy.get(tab_iterator) 
                    combined_df = combined_df.append(temp_df, ignore_index=True)
                    x+=1
                combined_df.to_csv('{}\{}.csv'.format(new_csv_folder_path, tab_iterator), index = False)
                print("Job # {}: CSV created for {}".format(t, tab_iterator))

        except AttributeError:
            print("Check that the tabs you want to join are named the same thing.")

        else:
            print("done :)")

In [ ]:
many_as_one(root_dir)